In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from utilsforecast.plotting import plot_series
from utilsforecast.evaluation import evaluate
from utilsforecast.losses import *

import warnings
warnings.filterwarnings('ignore')

In [ ]:
DATA_URL = "https://raw.githubusercontent.com/marcopeix/FoundationModelsForTimeSeriesForecasting/refs/heads/main/data/Walmart.csv"

df = pd.read_csv(DATA_URL)
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')

df.head()

In [ ]:
plot_series(df=df, id_col="Store", time_col="Date", target_col="Weekly_Sales", max_ids=6)

## Zero-shot forecasting

Make sure to `pip install autogluon` to access Chronos.

In [ ]:
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

In [ ]:
# Format Pandas dataframe to Gluon dataframe

display(ts_data.head())

In [ ]:
HORIZON = 13

# Initialize the TimeSeriesPredictor


In [ ]:
# Zero-shot forecasts


In [ ]:
preds_df = predictions.reset_index()

preds_df = preds_df.rename(columns={
    "item_id": "Store",
    "timestamp": "Date",
    "mean": "chronos-bolt-small",
    "0.1": "chronos-bolt-small-lo-80",
    "0.9": "chronos-bolt-small-hi-80"
})

preds_df = preds_df[["Store", "Date", "chronos-bolt-small", "chronos-bolt-small-lo-80", "chronos-bolt-small-hi-80"]]

preds_df.head()

In [ ]:
plot_series(
    df=df, 
    forecasts_df=preds_df,
    id_col="Store",
    time_col="Date", 
    target_col="Weekly_Sales",
    level=[80],
    max_ids=6,
)

## Cross-validation

In [ ]:
# Define a function for cross-validation in autogluon (current stable version doesn't have a cross-validation function at the time of recording)
def autogluon_cv(data, horizon, n_windows, step_size=None):
    # If step_size is None, use the horizon for non-overlapping windows

    # Initialize the predictor

    # List to store all results

    for i in range(n_windows):
        # Compute test offset

        # Train/test split

        # Get predictions
        
        # Rename columns
        preds_df = preds.reset_index()
        preds_df = preds_df.rename(columns={
            "item_id": "Store",
            "timestamp": "Date",
            "mean": "chronos-bolt-small",
            "0.1": "chronos-bolt-small-lo-80",
            "0.9": "chronos-bolt-small-hi-80"
        })

        # Rename columns from test_df to have actual values
        test_df = test.reset_index()
        test_df = test_df.rename(columns={
            "item_id": "Store",
            "timestamp": "Date",
            "target": "Weekly_Sales"
        })
        # Merge actual values with predictions

        # Filter columns
        preds_df = preds_df[["Store", "Date", "Weekly_Sales", "chronos-bolt-small", "chronos-bolt-small-lo-80", "chronos-bolt-small-hi-80"]]

        # Append dataframe

    # Concatenate all dataframes

    return all_preds_df

In [ ]:
# Run cross-validation

cv_df.head()

In [ ]:
plot_series(
    df=df, 
    forecasts_df=cv_df.drop(columns=["Weekly_Sales"]),
    id_col="Store",
    time_col="Date", 
    target_col="Weekly_Sales",
    level=[80],
    max_ids=6,
)

In [ ]:
eval_df = evaluate(
    pd.DataFrame(cv_df),
    metrics=[mae, smape],
    models=['chronos-bolt-small'],
    target_col='Weekly_Sales',
    id_col='Store',
    time_col="Date",
    agg_fn="mean"
)
eval_df.head()

## Forecasting with covariates

In [ ]:
# Create Gluon dataframe with features

display(ts_data_cov.head())

In [ ]:
# Add exogenous support in cross-validation
def autogluon_cv_exog(data, horizon, n_windows, step_size=None):
    if step_size is None:
        step_size = horizon

    all_preds = []

    for i in range(n_windows):
        test_end_offset = horizon + (i * step_size)
        train, test = data.train_test_split(test_end_offset)

        # Specify an external regressor
        

        # Get predictions
        
        # Rest of the function remains the same
        preds_df = preds.reset_index()
        preds_df = preds_df.rename(columns={
            "item_id": "Store",
            "timestamp": "Date",
            "mean": "chronos-bolt-small",
            "0.1": "chronos-bolt-small-lo-80",
            "0.9": "chronos-bolt-small-hi-80"
        })

        test_df = test.reset_index()
        test_df = test_df.rename(columns={
            "item_id": "Store",
            "timestamp": "Date",
        })
        preds_df = preds_df.merge(
            test_df[["Store", "Date", "Weekly_Sales"]],
            on=["Store", "Date"],
            how="left",
        )
        preds_df = preds_df[["Store", "Date", "Weekly_Sales", "chronos-bolt-small", "chronos-bolt-small-lo-80", "chronos-bolt-small-hi-80"]]

        all_preds.append(preds_df)

    all_preds_df = pd.concat(all_preds, ignore_index=True)

    return all_preds_df

In [ ]:
# Run cross-validation with exogenous features

exog_cv_df.head()

In [ ]:
plot_series(
    df=df, 
    forecasts_df=exog_cv_df.drop(columns=["Weekly_Sales"]),
    id_col="Store",
    time_col="Date", 
    target_col="Weekly_Sales",
    level=[80],
    max_ids=6,
)

In [ ]:
exog_eval_df = evaluate(
    pd.DataFrame(exog_cv_df),
    metrics=[mae, smape],
    models=['chronos-bolt-small'],
    target_col='Weekly_Sales',
    id_col='Store',
    time_col="Date",
    agg_fn="mean"
)
exog_eval_df.head()

## Fine-tuning

In [ ]:
# Add fine-tuning to cross-validation
def autogluon_cv_exog_finetune(data, horizon, n_windows, step_size=None, fine_tune=True):
    if step_size is None:
        step_size = horizon

    all_preds = []

    for i in range(n_windows):
        test_end_offset = horizon + (i * step_size)
        train, test = data.train_test_split(test_end_offset)

        # Specify fine-tuning parameters


        # Rest of the function remains the same
        preds = predictor.predict(train, known_covariates=test)
        preds_df = preds.reset_index()
        preds_df = preds_df.rename(columns={
            "item_id": "Store",
            "timestamp": "Date",
            "mean": "chronos-bolt-small",
            "0.1": "chronos-bolt-small-lo-80",
            "0.9": "chronos-bolt-small-hi-80"
        })

        test_df = test.reset_index()
        test_df = test_df.rename(columns={
            "item_id": "Store",
            "timestamp": "Date",
        })
        preds_df = preds_df.merge(
            test_df[["Store", "Date", "Weekly_Sales"]],
            on=["Store", "Date"],
            how="left",
        )
        preds_df = preds_df[["Store", "Date", "Weekly_Sales", "chronos-bolt-small", "chronos-bolt-small-lo-80", "chronos-bolt-small-hi-80"]]

        all_preds.append(preds_df)

    all_preds_df = pd.concat(all_preds, ignore_index=True)

    return all_preds_df

In [ ]:
# Cross-validation with fine-tuning

ft_exog_cv_df.head()

In [ ]:
ft_exog_eval_df = evaluate(
    pd.DataFrame(ft_exog_cv_df),
    metrics=[mae, smape],
    models=['chronos-bolt-small'],
    target_col='Weekly_Sales',
    id_col='Store',
    time_col="Date",
    agg_fn="mean"
)
ft_exog_eval_df.head()